In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense
from keras.applications import DenseNet121, DenseNet169, DenseNet201, ResNet50
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import pandas as pd
from tqdm import tqdm
import gc
import cv2

In [3]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [4]:
img_width, img_height = (224, 224)

train_data_dir = './data/train_img/'
test_data_dir = './data/test_img/'
epochs = 10
batch_size = 128

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [6]:
def get_renset50_model():
    resnet_model = ResNet50(include_top=False, input_shape=input_shape)
    # add a global spatial average pooling layer
    x = resnet_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=resnet_model.input, outputs=predictions)
    
    return model

def get_dense_201_model():
    densenet_201 = DenseNet201(include_top=False, input_shape=input_shape, weights='imagenet')
    # add a global spatial average pooling layer
    x = densenet_201.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=densenet_201.input, outputs=predictions)
    
    return model

def get_dense_169_model():
    densenet_169 = DenseNet169(include_top=False, input_shape=input_shape, weights='imagenet')
    
    x = densenet_169.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=densenet_169.input, outputs=predictions)
    
    return model

def get_dense_121_model():
    densenet_121 = DenseNet121(include_top=False, input_shape=input_shape, weights='imagenet')
    # add a global spatial average pooling layer
    x = densenet_121.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=densenet_121.input, outputs=predictions)
    
    return model

In [7]:
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv')
attributes = pd.read_csv('./data/attributes.txt', delimiter='\t', header=None, index_col=0)
classes = pd.read_csv('./data/classes.txt', delimiter='\t', header=None, index_col=0)

In [8]:
def get_imgs(src, df, labels = False):
    if labels == False:
        imgs = []    
        files = df['Image_name'].values
        for file in tqdm(files):
            im = cv2.imread(os.path.join(src, file))
            im = cv2.resize(im, (img_width, img_height))
            imgs.append(im)
        return np.array(imgs)
    else:
        imgs = []
        labels = []
        files = os.listdir(src)
        for file in tqdm(files):
            im = cv2.imread(os.path.join(src, file))
            im = cv2.resize(im, (img_width, img_height))
            imgs.append(im)
            labels.append(df.loc[file].values)
        return np.array(imgs), np.array(labels)

In [9]:
test_imgs = get_imgs(test_data_dir, test)

 21%|██        | 1128/5400 [00:09<00:37, 112.97it/s]/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 5400/5400 [00:48<00:00, 111.94it/s]


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_datagen.fit(test_imgs)

In [ ]:
model = get_dense_121_model()
model.load_weights('./models/densenets/densenet121_all_layers.h5')

In [ ]:
pred_dense_121 = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

In [ ]:
model = get_dense_169_model()
model.load_weights('./models/densenets/densenet169_all_layers.h5')

In [ ]:
pred_dense_169 = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

In [11]:
model = get_dense_201_model()
model.load_weights('./models/densenets/densenet201_all_layers.h5')

In [12]:
pred_dense_201 = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

11/11 [==============================] - 31s 3s/step


In [13]:
model = get_renset50_model()
model.load_weights('./models/resnet50/resnet_all_layers.h5')

In [14]:
pred_resnet50_201 = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

11/11 [==============================] - 17s 2s/step


In [15]:
# pred = 1.0/3.0 * (pred_dense_121 + pred_dense_169 + pred_dense_201)
pred = 1.0/2.0 * (pred_dense_201 + pred_resnet50_201)

In [16]:
sub = pd.read_csv('./data/sample_submission.csv')
sub.iloc[:, 1:] = pred.round().astype(int)
sub.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,1,0,1,0,1,1,0,0,...,0,1,0,1,0,0,0,0,1,0
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,0,1,1,1,0,0,...,0,1,1,1,0,0,0,0,0,1
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [17]:
sub.to_csv('submission.csv', index=False)

In [18]:
sub.shape

(5400, 86)